## iwls function directly derived from 
https://github.com/pysal/spglm/blob/master/spglm/iwls.py

In [21]:
import numpy as np
from scipy import linalg
import numpy.linalg as la
from scipy import sparse as sp
from scipy.sparse import linalg as spla
from spreg.utils import spdot, spmultiply
from scipy import special
import libpysal as ps

In [6]:
FLOAT_EPS = np.finfo(float).eps

### Fitting the MGWR model for the updated/adjusted dependent variables

#### This should remain the same as the GWR calculation given the partial smooth definition for process j (in this case the only 1) given in:
Yu, H., Fotheringham, S., Li, Z., Oshan, T., Kang, W., & Wolf, L. J. (2018). Inference in multiscale geographically weighted regression. doi:10.31219/osf.io/4dksb

\begin{align}
\hat{\beta} & _{ij}= e_j (X^T W_iX)^{-1} X^TW_i y \\
\end{align}

In [3]:
def compute_betas_mgwr(y,x,wi):
    xT = (x*wi).T
    xtx = np.dot(xT,x)
    xtx_inv_xt = linalg.solve(xtx, xT)
    betas = np.dot(xtx_inv_xt, y)
    return betas, xtx_inv_xt

In [ ]:
def compute_betas_mgwr(y,x,wi):
    xT = (x*wi).T
    xtx = np.dot(xT,x)
    xtx_inv_xt = linalg.solve(xtx, xT)
    betas = np.dot(xtx_inv_xt, y)
    return betas, xtx_inv_xt

#### I am confused if this is correct and remains the same as in GWPR and how to go about handling it for more than one dependent variables

### Only keeping the calculations for the log link function for now

In [39]:
def deriv (mu):
    p = np.clip(mu, FLOAT_EPS, np.inf)
    deriv_p = 1.0/p
    return deriv_p

def starting_mu(y):
    return (y+y.mean())/2
    
def weights(mu):
    p = np.clip(mu, FLOAT_EPS, np.inf)
    deriv_p = 1.0/p
    variance = np.ones(mu.shape,np.float64)
    return 1.0/(deriv_p**2 * variance)
    
def predict(mu):
    x = np.clip(mu, FLOAT_EPS, np.inf)
    return np.log(x)

def fitted(lin_pred):
    fits = np.exp(lin_pred)
    return fits

### Calling the weights function from below to calculate weights at each i

In [ ]:
wi = Kernel.ker(i=5,data=coords,bw=50,function='exponential')

### The function to do the Fischer scoring algorithm

In [143]:
def iwls(y, x, offset,
         ini_betas=None, tol=1.0e-06, max_iter=200, wi=None):
    
    n = x.shape[0]
    n_iter= 0
    diff = 1.0e+06
    
    if ini_betas is None:
        betas = np.zeros((x.shape[1],1), np.float)
    else:
        betas = ini_betas
        
    y_off = y/offset
    y_off = starting_mu(y_off)
    v = predict(y_off)
    mu = starting_mu(y)
        
    while diff > tol and n_iter < max_iter:
        n_iter +=1
        w = weights(mu)
        z = v + deriv(mu) * (y-mu)
        w = np.sqrt(w)
        
        if not isinstance(x, np.ndarray):
            w = sp.csr_matrix(w)
            z = sp.csr_matrix(z)
            
        wx = spmultiply(x,w,array_out=False)
        wz = spmultiply(z,w,array_out=False)
        
        n_betas, xtx_inv_xt = compute_betas_mgwr(wz, wx, wi)
        print(n_betas)
        v = spdot(x, n_betas)
        mu = fitted(v)
        mu = mu * offset
        
        diff = min(abs(n_betas - betas))
        #print(diff)
        betas = n_betas
        
    return betas, mu, v, w, z, xtx_inv_xt, n_iter      

### Trying with data

In [254]:
data_p = ps.io.open(ps.examples.get_path('Tokyomortality.csv'))
coords = list(zip(data_p.by_col('X_CENTROID'),data_p.by_col('Y_CENTROID')))
off = np.array(data_p.by_col('eb2564')).reshape((-1,1))
y = np.array(data_p.by_col('db2564')).reshape((-1,1)) 
occ = np.array(data_p.by_col('OCC_TEC')).reshape((-1,1))
own = np.array(data_p.by_col('OWNH')).reshape((-1,1))
pop = np.array(data_p.by_col('POP65')).reshape((-1,1))
unemp = np.array(data_p.by_col('UNEMP')).reshape((-1,1))
#X = np.hstack([occ,own,pop,unemp])
x = occ

In [255]:
#X_std = (X-X.mean(axis=0))/X.std(axis=0)
x_std = (x-x.mean(axis=0))/x.std(axis=0)
y_std = (y-y.mean(axis=0))/y.std(axis=0)
off_std = (off-off.mean(axis=0))/off.std(axis=0)

In [144]:
rslt=iwls(y=y_std,x=x_std,offset=off_std,wi=wit,tol=1.0e-06, max_iter=200)

[[0.17503723]]
[[-0.05613545]]
[[-0.11840296]]
[[-0.12554567]]
[[-0.12613708]]
[[-0.12618409]]
[[-0.12618782]]
[[-0.12618811]]


In [257]:
#do the result loop for each i 
#map the parameters back after

#### Manually constructed weights for testing initially

In [ ]:
wi = np.diagonal(w)
wit=wi.reshape(-1,1)

### Limited functionality Kernel function for constructing weights for each point

In [215]:
def _kernel_funcs(zs,function):
    # functions follow Anselin and Rey (2010) table 5.4
    if function == 'triangular':
        return 1 - zs
    elif function == 'uniform':
        return np.ones(zi.shape) * 0.5
    elif function == 'quadratic':
        return (3. / 4) * (1 - zs**2)
    elif function == 'quartic':
        return (15. / 16) * (1 - zs**2)**2
    elif function == 'gaussian':
        return np.exp(-0.5 * (zs)**2)
    elif function == 'bisquare':
        return (1 - (zs)**2)**2
    elif function == 'exponential':
        return np.exp(-zs)
    else:
        print('Unsupported kernel function', function)

In [248]:
import numpy as np
#adaptive specifications should be parameterized with nn-1 to match original gwr
#implementation. That is, pysal counts self neighbors with knn automatically.

#Soft dependency of numba's njit
try:
    from numba import njit
except ImportError:

    def njit(func):
        return func


@njit
def local_cdist(coords_i, coords, spherical):
    """
    Compute Haversine (spherical=True) or Euclidean (spherical=False) distance for a local kernel.
    """
    if spherical:
        dLat = np.radians(coords[:, 1] - coords_i[1])
        dLon = np.radians(coords[:, 0] - coords_i[0])
        lat1 = np.radians(coords[:, 1])
        lat2 = np.radians(coords_i[1])
        a = np.sin(
            dLat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dLon / 2)**2
        c = 2 * np.arcsin(np.sqrt(a))
        R = 6371.0
        return R * c
    else:
        return np.sqrt(np.sum((coords_i - coords)**2, axis=1))


class Kernel(object):
    """
    GWR kernel function specifications.
    
    """

    def ker(i, data, bw, function, ids=None, points=None, fixed=True,
                 eps=1.0000001, spherical=False):
        
        dvec = np.sqrt(np.sum((np.array(coords_i) - np.array(coords))**2, axis=1)).reshape(-1)

        bandwidth = float(bw)

        kernel = _kernel_funcs(dvec / bandwidth,function)

        if function == "bisquare":  #Truncate for bisquare
            kernel[(dvec >= bandwidth)] = 0
            
        return kernel

### Extra

In [46]:
import sys
sys.path.append("/Users/msachde1/Desktop/Seattle_files/mgwr-optim")

In [51]:
from mgwr.gwr import GWR
import numpy as np
from spglm.family import Gaussian, Binomial, Poisson
from mgwr.gwr import MGWR
from mgwr.sel_bw import Sel_BW
import multiprocessing as mp
pool = mp.Pool()

In [52]:
bw = Sel_BW(coords, y_std, x_std).search(pool=pool)

In [56]:
model_gwr=GWR(coords, y_std, x_std, bw=bw, fixed=False, kernel='bisquare')

In [57]:
results_gwr=model_gwr.fit()

In [69]:
w=results_gwr.W

In [43]:
def build_wi (i, bw):
    wi = Kernel(i, coords, bw, fixed, function, points, spherical).kernel
    return wi        

In [ ]:
fixed = True
spherical = False
function = 'bisquare'

In [ ]:
rslt = iwls(y, X, offset, ini_params, fit_params, max_iter, wi=wi)

In [ ]:
rslt_f = map()